In [1]:
import pandas as pd

### Extracts Table S1.2

In [2]:
FILE = "../data/st98-2025-prices-and-capital-expenditure-data.xlsx"
raw = pd.read_excel(FILE, sheet_name="Tables", header=None)
raw

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Table S1.1 Crude oil prices,NaN,NaN,NaN,NaN,NaN,NaN,Table S1.2 Natural gas prices,NaN,NaN,NaN,NaN,NaN,NaN,Table S1.3 Value of annual Alberta energy res...,NaN,NaN,NaN,NaN
3,NaN,2023.00,2024.00,2025.0,2026.0,2034.0,NaN,NaN,2023.00,2024.00,2025.00,2026.00,2034.00,NaN,NaN,2024.00,2025.00,2026.00,2034.00
4,West Texas Intermediate (US$/bbl)a,77.62,75.72,66.0,68.0,76.5,NaN,Henry Hub price (US$/MMBtu)a,2.67,2.41,3.80,3.90,4.59,NaN,Nonupgraded bitumen,49338.03,49024.05,51217.99,58321.81
5,Western Canadian Select (US$/bbl)a,58.97,60.99,55.0,56.0,63.5,NaN,AECO-C price (Cdn$/GJ)b,2.74,1.45,2.71,3.82,4.37,NaN,Upgraded bitumen,46411.16,45323.81,45666.50,48685.51
6,a US$/bbl = U.S. dollars per barrel.,NaN,NaN,NaN,NaN,NaN,NaN,a US$/MMBtu = U.S. dollars per million British...,NaN,NaN,NaN,NaN,NaN,NaN,Crude oil,18154.47,17680.95,18616.00,17781.13
7,Historical values from the Government of Alber...,NaN,NaN,NaN,NaN,NaN,NaN,b Cdn$/GJ = Canadian dollars per gigajoule.,NaN,NaN,NaN,NaN,NaN,NaN,Natural gas liquids,19669.01,22570.57,24334.99,27615.36
8,Forecast have been rounded.,NaN,NaN,NaN,NaN,NaN,NaN,Historical values from the Canadian Gas Price ...,NaN,NaN,NaN,NaN,NaN,NaN,Marketable natural gas,5030.28,10991.39,16261.50,18430.09
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Total,138602.95,145590.77,156096.98,170833.90


In [3]:
natural_gas_price_table = raw.iloc[:, 7:13].loc[3:5].reset_index(drop=True)
natural_gas_price_table.columns = natural_gas_price_table.iloc[0]
natural_gas_price_table.drop([0], inplace=True)
natural_gas_price_table = natural_gas_price_table.rename(columns={natural_gas_price_table.columns[0]: "label"})
natural_gas_price_table

,label,2023.0,2024.0,2025.0,2026.0,2034.0
1,Henry Hub price (US$/MMBtu)a,2.67,2.41,3.80,3.90,4.59
2,AECO-C price (Cdn$/GJ)b,2.74,1.45,2.71,3.82,4.37


In [4]:
tidy = natural_gas_price_table.melt(
    id_vars="label",
    var_name="year",
    value_name="price"
)

tidy["year"] = tidy["year"].astype(int)
tidy["price"] = tidy["price"].astype(float)

tidy

,label,year,price
0,Henry Hub price (US$/MMBtu)a,2023,2.67
1,AECO-C price (Cdn$/GJ)b,2023,2.74
2,Henry Hub price (US$/MMBtu)a,2024,2.41
3,AECO-C price (Cdn$/GJ)b,2024,1.45
4,Henry Hub price (US$/MMBtu)a,2025,3.80
5,AECO-C price (Cdn$/GJ)b,2025,2.71
6,Henry Hub price (US$/MMBtu)a,2026,3.90
7,AECO-C price (Cdn$/GJ)b,2026,3.82
8,Henry Hub price (US$/MMBtu)a,2034,4.59
9,AECO-C price (Cdn$/GJ)b,2034,4.37


In [5]:
def classify(row):
    if "Henry" in row:
        return "Henry Hub", "USD/MMBtu"
    elif "AECO" in row:
        return "AECO-C", "CAD/GJ"
    else:
        return None, None


tidy[["hub", "unit"]] = tidy["label"].apply(
    lambda x: pd.Series(classify(x))
)
tidy = tidy.drop(columns="label")

tidy

,year,price,hub,unit
0,2023,2.67,Henry Hub,USD/MMBtu
1,2023,2.74,AECO-C,CAD/GJ
2,2024,2.41,Henry Hub,USD/MMBtu
3,2024,1.45,AECO-C,CAD/GJ
4,2025,3.80,Henry Hub,USD/MMBtu
5,2025,2.71,AECO-C,CAD/GJ
6,2026,3.90,Henry Hub,USD/MMBtu
7,2026,3.82,AECO-C,CAD/GJ
8,2034,4.59,Henry Hub,USD/MMBtu
9,2034,4.37,AECO-C,CAD/GJ


In [6]:
cleaned_data = tidy.sort_values(["hub", "year"]).reset_index(drop=True)

cleaned_data

,year,price,hub,unit
0,2023,2.74,AECO-C,CAD/GJ
1,2024,1.45,AECO-C,CAD/GJ
2,2025,2.71,AECO-C,CAD/GJ
3,2026,3.82,AECO-C,CAD/GJ
4,2034,4.37,AECO-C,CAD/GJ
5,2023,2.67,Henry Hub,USD/MMBtu
6,2024,2.41,Henry Hub,USD/MMBtu
7,2025,3.80,Henry Hub,USD/MMBtu
8,2026,3.90,Henry Hub,USD/MMBtu
9,2034,4.59,Henry Hub,USD/MMBtu


In [8]:
cleaned_data.to_csv("../data/2025_aer_gas_prices.csv", index=False)